<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/webrtc_audio_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(8000)"))


In [ ]:
!cd /content && rm -rf achatbot && git clone --recursive https://github.com/ai-bot-pro/achatbot.git

In [ ]:
%cd /content/achatbot

In [ ]:
!bash scripts/pypi_achatbot.sh dev

In [ ]:
!pip install -q "dist/achatbot-0.0.7.2-py3-none-any.whl[fastapi_bot_server]"

In [ ]:
# https://m08wf4l3jc-496ff2e9c6d22116-8000-colab.googleusercontent.com/docs
# for http api server on colab
!python -m src.cmd.http.server.fastapi_daily_bot_serve --port 8000

# Run webRTC Room AI Chat Bot

In [ ]:
!pip install -q "dist/achatbot-0.0.7.2-py3-none-any.whl[fastapi_bot_server,livekit,livekit-api,daily,silero_vad_analyzer,sense_voice_asr,llm_processor,tts_edge]"

In [ ]:
!huggingface-cli download FunAudioLLM/SenseVoiceSmall  --local-dir ./models/FunAudioLLM/SenseVoiceSmall --local-dir-use-symlinks False

In [ ]:
from google.colab import userdata
DAILY_API_KEY=userdata.get('DAILY_API_KEY')

LIVEKIT_URL=userdata.get('LIVEKIT_URL')
LIVEKIT_API_KEY=userdata.get('LIVEKIT_API_KEY')
LIVEKIT_API_SECRET=userdata.get('LIVEKIT_API_SECRET')

TOGETHER_API_KEY=userdata.get('TOGETHER_API_KEY')


run bot task woker with bot.json, e.g.: dummy_bot.json

- use daily/livekit room stream, u can click bot joined the room url, to start chat with bot with audio and camera stream,
  - [daily](https://www.daily.co/) need DAILY_API_KEY
  - [livekit](https://livekit.io/) need project url LIVEKIT_URL, LIVEKIT_API_KEY, LIVEKIT_API_SECRET
- use openai/groq/together.ai api llm model need api key
  - [openai](https://openai.com/) OPENAI_API_KEY
  - [groq](https://groq.com/) GROQ_API_KEY
  - [together.ai](https://www.together.ai/) TOGETHER_API_KEY




In [44]:
!cat /content/daily_bot.json

{
  "chat_bot_name": "DailyBot",
  "room_name": "chat-room",
  "room_url": "",
  "token": "",
  "room_manager": {
    "tag": "daily_room",
    "args": {
      "privacy": "public"
    }
  },
  "services": {
    "pipeline": "achatbot",
    "vad": "silero",
    "asr": "sense_voice",
    "llm": "groq",
    "tts": "edge"
  },
  "config": {
    "vad": {
      "tag": "silero_vad_analyzer",
      "args": { "stop_secs": 0.7 }
    },
    "asr": {
      "tag": "sense_voice_asr",
      "args": {
        "language": "zn",
        "model_name_or_path": "./models/FunAudioLLM/SenseVoiceSmall"
      }
    },
    "llm": {
      "tag": "openai_llm_processor",
      "base_url": "https://api.together.xyz/v1",
      "model": "Qwen/Qwen2-72B-Instruct",
      "language": "zh",
      "messages": [
        {
          "role": "system",
          "content": "你是一名叫奥利给的智能助理。保持回答简短和清晰。请用中文回答。"
        }
      ]
    },
    "tts": {
      "tag": "tts_edge",
      "args": {
        "voice_name": "zh-CN-YunjianNeural",

In [ ]:
!DAILY_API_KEY=$DAILY_API_KEY TOGETHER_API_KEY=$TOGETHER_API_KEY \
  python -m src.cmd.bots.main -f /content/daily_bot.json

In [45]:
!cat /content/livekit_bot.json

{
  "chat_bot_name": "LivekitBot",
  "room_name": "chat-room",
  "room_url": "",
  "token": "",
  "room_manager": {
    "tag": "livekit_room",
    "args": {
      "bot_name": "LivekitBot",
      "is_common_session": false
    }
  },
  "services": {
    "pipeline": "achatbot",
    "vad": "silero",
    "asr": "sense_voice",
    "llm": "groq",
    "tts": "edge"
  },
  "config": {
    "vad": {
      "tag": "silero_vad_analyzer",
      "args": { "stop_secs": 0.7 }
    },
    "asr": {
      "tag": "sense_voice_asr",
      "args": {
        "language": "zn",
        "model_name_or_path": "./models/FunAudioLLM/SenseVoiceSmall"
      }
    },
    "llm": {
      "tag": "openai_llm_processor",
      "base_url": "https://api.together.xyz/v1",
      "model": "Qwen/Qwen2-72B-Instruct",
      "language": "zh",
      "messages": [
        {
          "role": "system",
          "content": "你是一名叫奥利给的智能助理。保持回答简短和清晰。请用中文回答。"
        }
      ]
    },
    "tts": {
      "tag": "tts_edge",
      "args": {
 

In [ ]:
!LIVEKIT_URL=$LIVEKIT_URL LIVEKIT_API_KEY=$LIVEKIT_API_KEY LIVEKIT_API_SECRET=$LIVEKIT_API_SECRET TOGETHER_API_KEY=$TOGETHER_API_KEY \
  LOG_LEVEL=debug python -m src.cmd.bots.main -f /content/livekit_bot.json

In [ ]:
from google.colab import userdata
REDIS_PASSWORD=userdata.get('REDIS_PASSWORD')

In [ ]:
!LOG_LEVEL=debug REDIS_PASSWORD=$REDIS_PASSWORD python -m src.cmd.bots.main -f /content/task_bot.json